In [1]:
import numpy as np
import scipy.optimize as sp
import QuantLib as ql
import yfinance as yf
import datetime

In [4]:
alpha = 0.3
beta = 0.04
gamma = 0.02
r0 = 0.054
F=1
T_bond=5#maturity of t-bill
n_coup=10#number of coupons paid by this t-bill

In [16]:
def P0_vasicek(rt,t,T):
    A_t_T=1/alpha*(1-np.exp(-alpha*(T-t)))
    B_t_T=(beta-beta**2/2/alpha**2)*(A_t_T-(T-t))-gamma**2*A_t_T**2/4/alpha
    P_t_T=np.exp(-rt*A_t_T+B_t_T)
    y_t_T=-np.log(P_t_T)/(T-t)
    # print(rt,A_t_T,B_t_T,P_t_T,y_t_T)
    return P_t_T

def Pc_vasicek(F,rt,t,T,y,n):
    # print(y)
    val=F*P0_vasicek(rt,t,T)#final payment with discount
    for i in range(t+1,n+1):
        val+=y*F*P0_vasicek(rt,t,i*T/n)
    return val

In [17]:

simulate_yield_vasicek= sp.newton(lambda y:Pc_vasicek(1,r0,0,T_bond,y,n_coup)-F,0.05)
r_simulated=simulate_yield_vasicek*2

0.04419963749443983

In [25]:
face=100000; T_frwd=1; N=10000; h=0.001;T_call=1
np.random.seed(N)

In [90]:
r_list=[]
for n in range(N):
    k = int(T_frwd/h+1)
    r_t = [r0] * k
    Z = np.random.normal(0,1,k-1)
    for i in range(1,k):
        r_t[i] = r_t[i-1] + alpha * (beta - r_t[i-1])*h + gamma*np.sqrt(h)*Z[i-1]
    r_list.append(r_t[-1])
r_hat=np.mean(r_list)
r_hat

0.050736422170673146

In [93]:
note=face* P0_vasicek(r_hat,t=1,T=6)
for i in range(3,11):
    coupon=r_hat*face*P0_vasicek(r_hat,3,i/2)
    note+=coupon
NOTE_PV=note*np.exp(-r0*1)
note,NOTE_PV

C:\Users\Julia\AppData\Local\Temp\ipykernel_20960\2155808784.py:5: RuntimeWarning: invalid value encountered in scalar divide
  y_t_T=-np.log(P_t_T)/(T-t)


(121451.67704308179, 115067.21821910307)

In [103]:
def MCVasicekCall(F, K, Tc, Tb, n_coup, h, N):
    np.random.seed(2024)  # Set random seed for reproducibility
    bond_P = []  # List to store bond prices
    for n in range(N):
        k = int(Tc / h + 1)  # Number of time steps
        r_t = [r0] * k  # Initialize the short-term interest rate path
        Z = np.random.normal(0, 1, k - 1)  # Generate random normal variables
        for i in range(1, k):
            r_t[i] = r_t[i - 1] + alpha * (beta - r_t[i - 1]) * h + gamma * np.sqrt(h) * Z[i - 1]  # Update short-term interest rate path
        r_hat = np.mean(r_t)
        # print(r_hat)# Calculate the average short-term interest rate
        # Calculate bond price at Tc and apply discount factor
        bond_price = max(0, Pc_vasicek(F, r_t[-1], Tc, Tb, simulate_yield_vasicek*2, n_coup) - K) * np.exp(-r_hat * Tc)
        # print(bond_price)
        bond_P.append( bond_price)  # Append the bond price to the list
    return bond_P


In [104]:
call_series=MCVasicekCall(F,note,T_call,T_bond,n_coup,h,N)
price=np.mean(call_series)
price_std=np.std(call_series)

C:\Users\Julia\AppData\Local\Temp\ipykernel_20960\2155808784.py:5: RuntimeWarning: invalid value encountered in scalar divide
  y_t_T=-np.log(P_t_T)/(T-t)


In [106]:
from scipy.stats import norm
def confidence_interval(mean, std, n):
    z = norm.ppf(0.975)  # 95% confidence interval
    margin_error = z * (std / np.sqrt(n))
    lower_bound = mean - margin_error
    upper_bound = mean + margin_error
    return lower_bound, upper_bound
lower_bound, upper_bound = confidence_interval(price, price_std, 10000)
print(f"Mean Price:{price}\nStd Price:{price_std}\nFor {N} simulations: 95% Confidence Interval = ({lower_bound:.4f}, {upper_bound:.4f})")

Mean Price:368.7354722205034
Std Price:1137.6959680091113
For 10000 simulations: 95% Confidence Interval = (346.4370, 391.0339)
